In [1]:
import pandas as pd
from teradataml.dataframe.copy_to import copy_to_sql
from teradataml.context.context import *
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
import numpy as np

In [68]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [69]:
train = train.dropna()
test = test.fillna(0)

## Train Cykotine Model

In [74]:
regr = RandomForestRegressor(n_estimators=50,max_depth=25, random_state=0)
regr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=25,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=50,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

## Generate Cykotine Forecasted Scores

In [80]:
file = open("final_ds.csv","w")
file.write('Date,Patient,Encounter,cs_score_pred')
file.write('\n')
for i in X_test:
    pat_data = (i[:3])
    pred_res = regr.predict([i[-144:]])
    s = np.concatenate([pat_data,pred_res])
    for j in s:
        file.write(str(j)+",")
    file.write('\n')

In [13]:
test=pd.read_csv('final_ds.csv')
test = test.fillna(0)
X_test=test.values

## Preprocess Mortality Data

In [2]:
train=pd.read_csv('mort_train.csv')

In [4]:
train1 = train.loc[train['Mortality_Label'] == 0]
train2 = train.loc[train['Mortality_Label'] == 1]

In [5]:
train2 = train2.sort_values(by=['cs_score'])

In [6]:
train2 = train2.tail(1700)

In [7]:
train = train1.append(train2)

In [8]:
X_train = train.drop(['patient','Mortality_Label'],axis=1).values
y_train = train['Mortality_Label'].values

## Train Mortality Model

In [10]:
regr = RandomForestClassifier(n_estimators=50,max_depth=25, random_state=0)
regr.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=25, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

## Generate Mortality Probabilities

In [21]:
file = open("final_mort_ds_probs.csv","w")
file.write('Date,Patient,Encounter,mort_prob')
file.write('\n')
for i in X_test:
    pat_data = (i[:3])
    pred_res = regr.predict_proba([i[-1:]])
    s = np.concatenate([pat_data,[pred_res[0][1]]])
    for j in s:
        file.write(str(j)+",")
    file.write('\n')

KeyboardInterrupt: 

## Preprocess ICU Days Data

In [119]:
train=pd.read_csv('icu_train.csv')

In [120]:
train1 = train.loc[train['Days_In_ICU'] == 0]
train2 = train.loc[train['Days_In_ICU'] > 0]

In [121]:
train1 = train1.sort_values(by=['cs_score'])

In [122]:
train1 = train1.tail(1700)

In [123]:
train = train1.append(train2)

In [125]:
X_train = train.drop(['patient','Days_In_ICU'],axis=1).values
y_train = train['Days_In_ICU'].values

## Train ICU Days Model

In [129]:
regr = RandomForestRegressor(max_depth = 10, max_features = 'sqrt', n_estimators = 10)
regr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

## Generate ICU Days

In [ ]:
file = open("final_icu_ds_probs.csv","w")
file.write('Date,Patient,Encounter,icu_prob')
file.write('\n')
for i in X_test:
    pat_data = (i[:3])
    pred_res = regr.predict_proba([i[-1:]])
    s = np.concatenate([pat_data,[pred_res[0][1]]])
    for j in s:
        file.write(str(j)+",")
    file.write('\n')

## Preprocess Covid19 +/- Data

In [3]:
train=pd.read_csv('covid_model.csv')

In [4]:
train1 = train.loc[train['test_result'] == 1]
train2 = train.loc[train['test_result'] == 0]

In [5]:
train2 = train2.sort_values(by=['cs_score'])

In [6]:
train2 = train2.tail(1700)

In [7]:
train = train1.append(train2)

In [8]:
X_train = train.drop(['patient','test_result'],axis=1).values
y_train = train['test_result'].values

## Train Covid19 +/- Data

In [9]:
regr = RandomForestClassifier()
regr.fit(X_train, y_train)

/anaconda2/envs/py3Env/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## Generate Covid19 +/- Probabilities

In [ ]:
file = open("final_covid_ds_probs.csv","w")
file.write('Date,Patient,Encounter,covid_prob')
file.write('\n')
for i in X_test:
    pat_data = (i[:3])
    pred_res = regr.predict_proba([i[-1:]])
    s = np.concatenate([pat_data,[pred_res[0][1]]])
    for j in s:
        file.write(str(j)+",")
    file.write('\n')

## Preprocess Ventilator Data

In [2]:
train=pd.read_csv('vent_train.csv')

In [3]:
train1 = train.loc[train['Vent_Status'] == 1]
train2 = train.loc[train['Vent_Status'] == 0]

In [4]:
train2 = train2.sort_values(by=['cs_score'])

In [5]:
train2 = train2.tail(1700)

In [6]:
train = train1.append(train2)

In [7]:
X_train = train.drop(['patient','Vent_Status'],axis=1).values
y_train = train['Vent_Status'].values

## Train Ventilator Model

In [8]:
regr = RandomForestClassifier(n_estimators=50,max_depth=25, random_state=0)
regr.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=25, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

## Generate Ventilator Probabilities

In [ ]:
file = open("final_vent_ds_probs.csv","w")
file.write('Date,Patient,Encounter,vent_prob')
file.write('\n')
for i in X_test:
    pat_data = (i[:3])
    pred_res = regr.predict_proba([i[-1:]])
    s = np.concatenate([pat_data,[pred_res[0][1]]])
    for j in s:
        file.write(str(j)+",")
    file.write('\n')

## Preprocess Hospitalized Days Data

In [84]:
train=pd.read_csv('host_train.csv')

In [85]:
train1 = train.loc[train['Days_Hospitalized'] > 0]
train2 = train.loc[train['Days_Hospitalized'] == 0]

In [86]:
train2 = train2.sort_values(by=['cs_score'])

In [87]:
train2 = train2.tail(200)

In [88]:
train = train1.append(train2)

In [89]:
X_train = train.drop(['patient','Days_Hospitalized'],axis=1).values
y_train = train['Days_Hospitalized'].values

## Train Hospitalize Days Model

In [79]:
regr = RandomForestRegressor()
regr.fit(X_train, y_train)

/anaconda2/envs/py3Env/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

## Generate Hospitalized Days

In [ ]:
file = open("final_host_ds_probs.csv","w")
file.write('Date,Patient,Encounter,host_prob')
file.write('\n')
for i in X_test:
    pat_data = (i[:3])
    pred_res = regr.predict_proba([i[-1:]])
    s = np.concatenate([pat_data,[pred_res[0][1]]])
    for j in s:
        file.write(str(j)+",")
    file.write('\n')

## Parameter Tuning 

In [127]:
from sklearn.model_selection import GridSearchCV
estimator = RandomForestRegressor()
param_grid = { 
        "n_estimators"      : [10,50,100],
        "max_features"      : ["auto", "sqrt", "log2"],
        "max_depth" : [2,10,15]
        }
grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=5)

grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [2, 10, 15],
       